In [1]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset

dataset_path = "/media/gronkomatic/Embiggen/ai-stuff/datasets/OpenHermes-2.5/openhermes2_5.json"
model_path = "/media/gronkomatic/Embiggen/ai-stuff/training-results/runs/run-20240315-211134/checkpoint-70180"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.chat_template = "<s>{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}</s>"

In [16]:
dataset = load_dataset("json", data_files=dataset_path)
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'topic', 'model_name', 'hash', 'language', 'custom_instruction', 'id', 'conversations', 'source', 'system_prompt', 'category', 'skip_prompt_formatting', 'avatarUrl', 'title', 'views', 'model'],
        num_rows: 1001551
    })
})

In [17]:
dataset = dataset["train"].select(range(10))
dataset

Dataset({
    features: ['idx', 'topic', 'model_name', 'hash', 'language', 'custom_instruction', 'id', 'conversations', 'source', 'system_prompt', 'category', 'skip_prompt_formatting', 'avatarUrl', 'title', 'views', 'model'],
    num_rows: 10
})

In [19]:
# Define a function to rename the keys
def rename_keys(example):
    new_conversations = []
    for conv in example['conversations']:
        if conv["from"] == "gpt":
            conv["from"] = "assistant"
            
        new_conversations.append({
            "role": conv["from"],
            "content": conv["value"]
        })
    return {
        "conversations": new_conversations
    }


# Apply the function to the dataset
renamed_dataset = dataset.map(rename_keys)
renamed_dataset["conversations"]

[[{'content': 'Every day, a tree drops 7 leaves. How many leaves would it drop in a month of February in a non-leap year? Include your logic.',
   'role': 'human'},
  {'content': "Here's the logic behind this:\n\n1. We know that February has 28 days in a non-leap year.\n2. If the tree drops 7 leaves every day, then over the course of February, it would drop:\n   Leaves dropped in February = Leaves per day * Days in February\n   = 7 leaves * 28 days\n   = 196 leaves\n\nSo, the tree would drop 196 leaves in February in a non-leap year.",
   'role': 'assistant'}],
 [{'content': 'In analytical chemistry, what is the principle behind the use of an internal standard in quantitative analysis?\nA. It compensates for variations in sample preparation and instrumental response.\nB. It enhances the sensitivity of the analytical method.\nC. It reduces the detection limit of the analytical method.\nD. It increases the resolution between analyte peaks in chromatography.\nE. None of the above.',
   'r

In [20]:
dataset = renamed_dataset.map(lambda x: {"formatted_conversations": tokenizer.apply_chat_template(
    x["conversations"], tokenize=False, add_generation_prompt=False)})

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [21]:
print(dataset['formatted_conversations'][0])

<s><|im_start|>human
Every day, a tree drops 7 leaves. How many leaves would it drop in a month of February in a non-leap year? Include your logic.<|im_end|>
<|im_start|>assistant
Here's the logic behind this:

1. We know that February has 28 days in a non-leap year.
2. If the tree drops 7 leaves every day, then over the course of February, it would drop:
   Leaves dropped in February = Leaves per day * Days in February
   = 7 leaves * 28 days
   = 196 leaves

So, the tree would drop 196 leaves in February in a non-leap year.<|im_end|>
</s>
